## Purpose:
Analyze how to join the Reddit comments table with the posts table. The comments table does not contain the original post that started a particular thread.


## Initialize

In [2]:
# Utility
import sys,os
import time

import pandas as pd
pd.set_option('display.max_columns', 500) # more columns displayed at once
pd.options.display.max_colwidth = 200 # more of the text displayed at once

# Custom
import processing

## Import Data

**Data imported from Google's BigQuery:** https://bigquery.cloud.google.com/

For the data that we are interested in, i.e. posts under the subReddit 'AskDocs' along with comments on those posts. Reddit has seperated the data for the posts from the comments on those posts so we need to join the two.

In [2]:
# import comments
path_to_data = '../data/reddit_comments_askDocs_2014_to_2018_03.gz'
df_comments = load_comments(path_to_data)
df_comments.head(2)

Shape: (557648, 24)


,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,link_id_short,parent_id_short,post_id
0,"for a manlet such as yourself I'd recommend at least 70 oz of water daily and at least 7 hours of sleep. Cut down on red meat, smoking, and sodium intake and check back in a few days.",1.0,NaN,NaN,-Ai,This user has not yet been verified.,NaN,1513411674,t5_2xtuc,t3_7k5x2h,t3_7k5x2h,0,1.514772e+09,0,0,drbt2db,AskDocs,NaN,NaN,default,NaN,7k5x2h,7k5x2h,drbt2db
1,Thank you very much for answering!,1.0,NaN,NaN,-SY,This user has not yet been verified.,NaN,1445798103,t5_2xtuc,t3_3q697b,t1_cwcf958,2,1.447190e+09,0,0,cwcfjpr,AskDocs,2.0,NaN,default,NaN,3q697b,cwcf958,cwcfjpr


In [3]:
path_to_posts = '../data/original_posts_under_askDocs_subreddit_id.gz'
df_posts = pd.read_csv(path_to_posts,low_memory=False)
print('Shape',df_posts.shape)
df_posts.head(2)

Shape (43615, 34)


,domain,subreddit,selftext,saved,id,from_kind,gilded,from,stickied,title,num_comments,score,retrieved_on,over_18,thumbnail,subreddit_id,hide_score,link_flair_css_class,author_flair_css_class,downs,archived,is_self,from_id,permalink,name,created,url,author_flair_text,quarantine,author,created_utc,link_flair_text,ups,distinguished
0,self.AskDocs,AskDocs,\n Age: 28-32\n Sex: M\n Height: 6'\n Weight: 175\n Race: Cauc\n Duration of complaint: 1 year\n Location (Geographic and on body): pelvis / urethra\n Any existing relevant...,False,3e50jf,NaN,0,NaN,False,Pain in pelvic floor / during arousal / urinating / ejaculating (with cloudy pee pics),0,1,1440597166,False,self,t5_2xtuc,False,NaN,default,0,False,True,NaN,/r/AskDocs/comments/3e50jf/pain_in_pelvic_floor_during_arousal_urinating/,t3_3e50jf,1437528445,http://www.reddit.com/r/AskDocs/comments/3e50jf/pain_in_pelvic_floor_during_arousal_urinating/,This user has not yet been verified.,False,doctorplsrespond,1437524845,NaN,1,NaN
1,self.AskDocs,AskDocs,"27\nM\n6'0""\n170 lbs.\nWhite\n2+ weeks\nNorthern California\nno prior medical conditions\nno medications\n\nI feel like my armpits are a bit swollen but there is no pain. Even when pressure is app...",False,37x41u,NaN,0,NaN,False,What's wrong with me?,3,3,1440702925,False,self,t5_2xtuc,False,NaN,default,0,False,True,NaN,/r/AskDocs/comments/37x41u/whats_wrong_with_me/,t3_37x41u,1433045646,http://www.reddit.com/r/AskDocs/comments/37x41u/whats_wrong_with_me/,This user has not yet been verified.,False,dudeotd,1433042046,NaN,3,NaN


### Showing processing for joining comments to posts tables 
(This is implemented as a class in processing.py)

In [4]:
comment_post_ids = df_comments['link_id_short'].tolist()
post_ids = df_posts['id'].tolist()

print('Unique posts in post table:',len(set(post_ids)))
print('Unique posts in comment table:',len(set(comment_post_ids)))

# Get set of ids that are in both tables:
id_intersect = (set(post_ids) & set(comment_post_ids))
print('Unique posts also in comment table:', len(id_intersect) )

# Create table with only those intersect ids
df_intersect = df_posts.loc[df_posts['id'].isin(id_intersect)]

df_intersect = df_intersect.rename(index=str, columns={"selftext": "body"}).copy()

print('number of comments that had ids in posts also')
print(df_comments.loc[df_comments['link_id_short'].isin(id_intersect)].shape[0])

Unique posts in post table: 43615
Unique posts in comment table: 148406
Unique posts also in comment table: 30710


In [5]:
print("In posts table but not in comments table: \n",set(df_intersect.columns)-set(df_comments.columns) )
print()

print("In comments table but not in posts table: \n",set(df_comments.columns)-set(df_intersect.columns) )

columns_in_both = (set(df_comments.columns) & set(df_intersect.columns))
print('columns_in_both:',len(columns_in_both))
# update the columns we still want to import from posts table
columns_in_both.update(["title","url","over_18"])
columns_in_both = list(columns_in_both)

In posts table but not in comments table: 
 {'saved', 'title', 'over_18', 'url', 'link_flair_text', 'stickied', 'from', 'num_comments', 'created', 'from_id', 'quarantine', 'permalink', 'thumbnail', 'domain', 'from_kind', 'link_flair_css_class', 'is_self', 'hide_score'}

In comments table but not in posts table: 
 {'link_id', 'parent_id', 'post_id', 'link_id_short', 'controversiality', 'removal_reason', 'parent_id_short', 'score_hidden'}
columns_in_both: 16


In [10]:
# Get comments following threads that are in both posts and comments
df_comments = df_comments.loc[df_comments['link_id_short'].isin(id_intersect)].copy()
# Get final intersect table
df_intersect = df_comments.append(df_intersect[columns_in_both]).copy()


## Implemented as function

In [2]:
# import data
path_to_data = '../data/reddit_comments_askDocs_2014_to_2018_03.gz'
df_comments = load_comments(path_to_data)
path_to_posts = '../data/original_posts_under_askDocs_subreddit_id.gz'
df_posts = pd.read_csv(path_to_posts,low_memory=False)

## Get uniqueness
# Uniqueness among posts
comment_post_ids = df_comments['link_id_short'].tolist()
post_ids = df_posts['id'].tolist()

# Get set of ids that are in both tables:
id_intersect = (set(post_ids) & set(comment_post_ids))

# Create table with only those intersect ids
df_intersect = df_posts.loc[df_posts['id'].isin(id_intersect)]
df_intersect = df_intersect.rename(index=str, columns={"selftext": "body"}).copy()

# Deterimine which columns would be helpful in final table
columns_in_both = (set(df_comments.columns) & set(df_intersect.columns))
columns_in_both.update(["title","url","over_18"])
columns_in_both = list(columns_in_both)

# Get comments following threads that are in both posts and comments
df_comments = df_comments.loc[df_comments['link_id_short'].isin(id_intersect)].copy()
# Get final intersect table
df_intersect = df_comments.append(df_intersect[columns_in_both]).copy()
print('Final combined table shape:',df_intersect.shape)

Shape: (557648, 24)


In [2]:
data_instance = DataPipeline(comments_path = '../data/reddit_comments_askDocs_2014_to_2018_03.gz',
                            posts_path = '../data/original_posts_under_askDocs_subreddit_id.gz')

In [3]:
df_test = data_instance.load_full_thread()

Comments Table Shape: (557648, 24)
Posts table shape: (43615, 34)
Final combined table shape: (139535, 27)
